##Semantic image search with OpenAI's CLIP
Demo of [OpenAI's CLIP](https://openai.com/blog/clip/):
- built with [transformers](https://huggingface.co/transformers/) from
🤗 Hugging Face
- based on 25,000 images from [Unsplash](https://unsplash.com/) and 7,685 images from the [Movie Database (TMDB)](https://www.themoviedb.org/)
- inspired by [Unsplash Image Search](https://github.com/haltakov/natural-language-image-search) by Vladimir Haltakov and [Alph, The Sacred River](https://github.com/thoppe/alph-the-sacred-river) by Travis Hoppe

In [4]:
#@title ##← Click on the circled arrow and wait for up to 1 minute
!pip install -q transformers >/dev/null

import pandas as pd, numpy as np
import os
import urllib.request
from transformers import CLIPProcessor, CLIPTextModel, CLIPModel, logging

from IPython.display import display, Markdown, HTML, clear_output
import ipywidgets as widgets

logging.get_verbosity = lambda: logging.NOTSET
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clear_output()

def compute_text_embeddings(list_of_strings):
    inputs = processor(text=list_of_strings, return_tensors="pt", padding=True)
    return model.get_text_features(**inputs)

urllib.request.urlretrieve('https://huggingface.co/spaces/vivien/clip/raw/main/data.csv', 'data.csv')
urllib.request.urlretrieve('https://huggingface.co/spaces/vivien/clip/raw/main/data2.csv', 'data2.csv')
urllib.request.urlretrieve('https://huggingface.co/spaces/vivien/clip/resolve/main/embeddings-vit-base-patch32.npy', 'embeddings.npy')
urllib.request.urlretrieve('https://huggingface.co/spaces/vivien/clip/resolve/main/embeddings2-vit-base-patch32.npy', 'embeddings2.npy')

df = {0: pd.read_csv('data.csv'), 1: pd.read_csv('data2.csv')}
embeddings = {0: np.load('embeddings.npy'), 1: np.load('embeddings2.npy')}
for k in [0, 1]:
  embeddings[k] = np.divide(embeddings[k], np.sqrt(np.sum(embeddings[k]**2, axis=1, keepdims=True)))
source = {0: '\nSource: Unsplash', 1: '\nSource: The Movie Database (TMDB)'}

def get_html(url_list, height=200):
    html = "<div style='margin-top: 20px; display: flex; flex-wrap: wrap; justify-content: space-evenly'>"
    for url, title, link in url_list:
        html2 = f"<img title='{title}' style='height: {height}px; margin-bottom: 10px' src='{url}'>"
        if len(link) > 0:
            html2 = f"<a href='{link}' target='_blank'>" + html2 + "</a>"
        html = html + html2
        #print('html',html)
        print('limk',link)
        print('url', url)
        print('title', title)
    html += "</div>"
    return html

query = widgets.Text(layout=widgets.Layout(width='400px'))
dataset =widgets.Dropdown(
    options=['Unsplash', 'Movies'],
    value='Unsplash'
)
button = widgets.Button(description="Search")
output = widgets.Output()

display(widgets.HBox([query, button, dataset],
                     layout=widgets.Layout(justify_content='center')),
        output)

def image_search(query, n_results=24):
    text_embeddings = compute_text_embeddings([query]).detach().numpy()
    k = 0 if dataset.value == 'Unsplash' else 1
    results = np.argsort((embeddings[k]@text_embeddings.T)[:, 0])[-1:-n_results-1:-1]
    return [(df[k].iloc[i]['path'],
             df[k].iloc[i]['tooltip'] + source[k],
             df[k].iloc[i]['link']) for i in results]

def on_button_clicked(b):
    if len(query.value) > 0:
        results = image_search(query.value)
        output.clear_output()
        with output:
            display(HTML(get_html(results)))

button.on_click(on_button_clicked)
dataset.observe(on_button_clicked, names='value')

Output()

In [9]:
df = pd.read_csv('data.csv')
#df

def compute_text_embeddings(list_of_strings):
  inputs = processor(text=list_of_strings, return_tensors="pt", padding=True)
  return model.get_text_features(**inputs)

for i in df['path'] :
  print(i)

流式输出内容被截断，只能显示最后 5000 行内容。
https://images.unsplash.com/photo-1549215650-bf212238f04c?w=700
https://images.unsplash.com/photo-1550482306-5ff6743b1565?w=700
https://images.unsplash.com/photo-1553029230-2be4f5c23de8?w=700
https://images.unsplash.com/photo-1553117595-ce350239bde9?w=700
https://images.unsplash.com/photo-1553634551-f307dd5e69ab?w=700
https://images.unsplash.com/photo-1551593901-3b00af90a713?w=700
https://images.unsplash.com/photo-1551908325-9bd4c5c33171?w=700
https://images.unsplash.com/photo-1552529420-98576a9dc441?w=700
https://images.unsplash.com/photo-1551880080-989d0e3ff339?w=700
https://images.unsplash.com/photo-1552923410-f561a49581c4?w=700
https://images.unsplash.com/photo-1543599568-cdc16cc6c41b?w=700
https://images.unsplash.com/photo-1543866282-7205a4b88f09?w=700
https://images.unsplash.com/photo-1543965205-539391067959?w=700
https://images.unsplash.com/photo-1544048618-c372ffefdb1f?w=700
https://images.unsplash.com/photo-1543867702-37b3f7eabf84?w=700
https://image